# Utilities for Thesis
### Figure generators and such

# Generic Imports

In [ ]:
import os
import math
import numpy as np

import madmom
from madmom.audio.filters import LogarithmicFilterbank, MelFilterbank

from scipy.io import wavfile
from scipy.signal import spectrogram, get_window

import matplotlib.pyplot as plt

# Initialization

### Debugger

In [ ]:
debug = True

class Debugger:
    def __init__(self, debug):
        self.debug = debug
        
    def log(self, *msg):
        if(self.debug):
            print(*msg)
            
dbg = Debugger(debug)
dbg.log('debugger', 'initialized')

### Globals Variables and Configurations

In [ ]:
# audio file
AUDIO_FILE_NAME = 'zztop_badtothebone_mono_5sec.wav'
AUDIO_PATH_RELATIVE = '../../audio_for_thesis'
AUDIO_LEN = 5
FS = None
AUDIO_BUFFER = None

assert os.path.exists(AUDIO_PATH_RELATIVE), 'Folder doesnt exist: {}'.format(AUDIO_PATH_RELATIVE)

# figures
FIG_FOLDER = 'utilities_for_thesis_figures'
FIG_SIZE = (9,6)
DPI = 80

assert os.path.exists(FIG_FOLDER), 'Folder doesnt exist: {}'.format(FIG_FOLDER)

# framing
FRAMING_FRAME_SIZE = 256 #128
FRAMING_HOP_SIZE = 152 #76 #96
FRAMING_FRAME_N = 3

### Global Functions and Helpers

In [ ]:
def get_fig_save_path(fig_name):
    return os.path.join(FIG_FOLDER, fig_name)

# 0 <= n <= N-1
def hann(n, N):
    return 0.5 * (1 - math.cos((2 * math.pi * n) / (N - 1)))

v_hann = np.vectorize(hann)

### Load Audio

In [ ]:
def load_audio():
    audio_path = os.path.join(os.getcwd(), AUDIO_PATH_RELATIVE)
    audio_path_absolute = os.path.abspath(audio_path)
    assert os.path.exists(audio_path_absolute), 'Audio file path doesnt exist: {}'.format(audio_path_absolute)

    audio_path_full = os.path.join(audio_path_absolute, AUDIO_FILE_NAME)
    assert os.path.exists(audio_path_full), 'Audio file doesnt exist: {}'.format(audio_path_full)

    fs, audio_buffer = wavfile.read(audio_path_full)

    assert fs == 44100, 'Sampling rate should be 44100 Hz'
    assert len(audio_buffer.shape) == 1, 'Audio should be mono'
    assert len(audio_buffer) == AUDIO_LEN * fs, 'Audio should be exactly %d seconds long' %(AUDIO_LEN)

    dbg.log(audio_path_full)
    dbg.log('audio buffer shape:', audio_buffer.shape, 'sampling rate:', fs)
    
    return fs, audio_buffer

FS, AUDIO_BUFFER = load_audio()

# Figures

### Time Domain Audio Signal Representation

In [ ]:
def plot_time_domain(figsize, dpi, save_path):
    plt.figure(figsize=figsize)
    plt.plot(AUDIO_BUFFER/AUDIO_BUFFER.max())
    audio_ticks = np.arange(0, len(AUDIO_BUFFER), FS/2)
    plt.xticks(audio_ticks, ["{:.1f}".format(tick/FS) for tick in audio_ticks], rotation='45')
    plt.xlim([0, len(AUDIO_BUFFER)])
    plt.xlabel('Time (Seconds)')
    plt.ylabel('Amplitude (Normalized)');

    plt.tight_layout()
    plt.savefig(save_path, dpi=dpi)
    
plot_time_domain(FIG_SIZE, DPI, get_fig_save_path('time_domain.png'))

### Frequency Domain Audio Signal Representation

In [ ]:
def plot_frequency_domain(figsize, dpi, save_path):
    '''
    # Original:

    frame_size = 2048
    start_frame = 2*FS

    frame = AUDIO_BUFFER[start_frame:start_frame+frame_size]
    spectrum = np.fft.fft(frame)

    abs_spec_size = int(frame_size/2+1)
    abs_spec = np.abs(spectrum)[:abs_spec_size]

    fft_freqs = np.fft.fftfreq(frame_size)*FS
    freq_ticks = np.arange(0, abs_spec_size, 128, dtype=int)
    freq_tick_labels = ["{:.1f}Hz".format(np.abs(fft_freqs[idx])) for idx in freq_ticks]
    print(fft_freqs[1024])


    plt.plot(abs_spec/abs_spec.max())
    plt.xticks(freq_ticks, freq_tick_labels, rotation="45");
    '''

    frame_size = AUDIO_LEN * FS # 220500

    frame = AUDIO_BUFFER[0:frame_size] # take whole 5sec clip as frame
    spectrum = np.fft.fft(frame) # fft resolution is same as size of AUDIO_BUFFER (220500)

    abs_spec_size = int(frame_size/2+1) # half cause mirrored (110251)
    abs_spec = np.abs(spectrum)[:abs_spec_size] # absolute values of half

    # frequency bin centers [0...0.5,-0.5...-0]
    # result in requency values when multiplied by sampling rate (cause same as resolution) (same as specifying d=1/sampling_rate)
    # with 220500 values
    # will only be used until index half, cause they are then mirrored in the negative
    fft_freqs = np.fft.fftfreq(frame_size)*FS
    dbg.log(fft_freqs[110250])

    # indices to be used, only go until 110251, cause of mirroring
    # 110251 / 7 = 15750.14, round down to get 8 indices instead of 7
    freq_ticks = np.arange(0, abs_spec_size, 15750, dtype=int)
    # labels (with Hz as value) for the corresponding indices
    freq_tick_labels = ["{:.1f}".format(np.abs(fft_freqs[idx])) for idx in freq_ticks]

    plt.figure(figsize=figsize)
    plt.plot(abs_spec/abs_spec.max())
    plt.xticks(freq_ticks, freq_tick_labels, rotation="45")

    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude / Power (Normalized)')

    plt.tight_layout()
    plt.savefig(save_path, dpi=dpi)

    dbg.log('spectrum resolution:', abs_spec.shape)
    dbg.log('fft frequencies:', fft_freqs.shape)
    dbg.log('indices to be used for freq. values:', freq_ticks)
    dbg.log('frequencies at above indices taken from fft frequencies:', freq_tick_labels)
    
plot_frequency_domain(FIG_SIZE, DPI, get_fig_save_path('frequency_domain.png'))

### Framing (frame size, hop size)

In [ ]:
def plot_signal_before_framing(figsize, dpi, save_path):
    frame_size = FRAMING_FRAME_SIZE
    hop_size = FRAMING_HOP_SIZE
    frame_n = FRAMING_FRAME_N

    start_sample = int(0.5*FS)

    audio_segment = AUDIO_BUFFER[start_sample: start_sample + ((frame_n-1)*(hop_size)) + frame_size]
    dbg.log('segment length: {} samples'.format(audio_segment.size))
    
    # calculate signal frames
    frames = []
    frames_start = []
    frames_end = []
    for i in range(0, frame_n):
        start_curr = start_sample + i * hop_size
        f = AUDIO_BUFFER[start_curr: start_curr +  frame_size]
        dbg.log('{} frame size: {} samples'.format(i+1, f.size))
        frames.append(f)
        frames_start.append(i*hop_size)
        frames_end.append((i*hop_size + frame_size)-1)
    
    # plot signal and highlight overlap
    plt.figure(figsize=(figsize[0], 0.5*figsize[1]))
    plt.plot(audio_segment/audio_segment.max())

    plt.axvspan(hop_size, frame_size-1, color='red', alpha=0.15)
    plt.axvspan(2*hop_size, (frame_size + hop_size)-1, color='red', alpha=0.15)

    # X-Axis and Y-Axis
    audio_ticks = np.arange(0, audio_segment.size, int(audio_segment.size/9))
    plt.xticks(audio_ticks, ["{:.0f}".format(tick+1) for tick in audio_ticks], rotation='45')
    plt.xlim([0, audio_segment.size])
    plt.xlabel('Sample')
    plt.ylabel('Amplitude');

    # top X-Axis
    ax_overlap = plt.twiny()
    overlap_ticks = np.concatenate((frames_start, frames_end))
    ax_overlap.set_xticks(overlap_ticks)
    overlap_labels = np.array([((i%3)+1, tick+1) for i, tick in enumerate(overlap_ticks)])
    ax_overlap.set_xticklabels(["F{}: {:.0f}".format(frame_n, sample) for frame_n, sample in (overlap_labels)])
    ax_overlap.set_xlim([0, audio_segment.size])

    plt.tight_layout()
    plt.savefig(save_path, dpi=dpi)
    
    x_start, x_end, y_start, y_end = plt.axis()
    
    return audio_segment, frames, x_start, x_end, y_start, y_end, overlap_ticks, overlap_labels
    
FRAMING_AUDIO_SEGMENT, FRAMING_FRAMES, FRAME_X_AX_START, FRAME_X_AX_END, FRAME_Y_AX_START, FRAME_Y_AX_END, \
    FRAMING_OVERLAP_TICKS, FRAMING_OVERLAP_LABELS \
    = plot_signal_before_framing(FIG_SIZE, DPI, get_fig_save_path('signal_before_framing.png'))

In [ ]:
def plot_signal_after_framing(figsize, dpi, save_path, audio_segment, frames, \
                              x_start, x_end, y_start, y_end, overlap_ticks, overlap_labels, smoothing = False):
    frame_size = FRAMING_FRAME_SIZE
    hop_size = FRAMING_HOP_SIZE
    frame_n = FRAMING_FRAME_N

    fig, ax = plt.subplots(3, 1, figsize=(figsize[0], 1.5*figsize[1]))

    # 1 frame
    for i in range(0, frame_n):

        # plot 1 frame and highlight overlap
        f = frames[i]

        if(smoothing):
            sample_idxs = np.arange(0, frame_size)
            smoothing_coefficients = v_hann(sample_idxs, frame_size)
            f = smoothing_coefficients * f

        f_norm = f/audio_segment.max()
        frame_in_segment = np.full(audio_segment.size, None)
        frame_in_segment[i*hop_size :i*hop_size + f_norm.size] = f_norm

        ax[i].plot(frame_in_segment)
        if i != 2:
            ax[i].axvspan(hop_size, f.size-1, color='red', alpha=0.15)
        if i != 0:
            ax[i].axvspan(2*hop_size, (f.size + hop_size)-1, color='red', alpha=0.15)

        #ax[i].set_title('Frame {}'.format(i+1))

        # labels, limits and ticks
        if i == 2:
            ax[i].set_xlabel('Sample')
        ax[i].set_ylabel('Amplitude');

        ax[i].set_xlim([x_start, x_end])
        ax[i].set_ylim([y_start, y_end])

        if i == 2:
            audio_ticks = np.arange(0, audio_segment.size, int(audio_segment.size/9))
            ax[i].set_xticks(audio_ticks)
            ax[i].set_xticklabels(["{:.0f}".format(tick+1) for tick in audio_ticks], rotation=45)
        else:
            ax[i].set_xticks([])

        # top X-Axes
        ax_x2 = ax[i].twiny()

        lower_bound = i*hop_size <= overlap_ticks
        upper_bound = overlap_ticks <= (i*hop_size) + frame_size
        bound = lower_bound * upper_bound
        ax_x2.set_xticks(overlap_ticks[bound])

        lower_bound = i*hop_size <= overlap_labels[:,1]
        upper_bound = overlap_labels[:,1] <= (i*hop_size) + frame_size
        bound = lower_bound * upper_bound
        ax_x2.set_xticklabels(["F{}: {:.0f}".format(frame_n, sample) for frame_n, sample in overlap_labels[bound]])

        ax_x2.set_xlim([x_start, x_end])

        # right Y-Axes
        ax_y2 = ax[i].twinx()
        ax_y2.set_yticks([])
        ax_y2.set_ylabel('Frame {}'.format(i+1), labelpad=10);

    plt.subplots_adjust(hspace=0.25)
    fig.tight_layout()

    plt.show()
    fig.savefig(save_path, dpi=dpi)

plot_signal_after_framing(FIG_SIZE, DPI, get_fig_save_path('signal_after_framing.png'), \
    FRAMING_AUDIO_SEGMENT, FRAMING_FRAMES, FRAME_X_AX_START, FRAME_X_AX_END, FRAME_Y_AX_START, FRAME_Y_AX_END, \
    FRAMING_OVERLAP_TICKS, FRAMING_OVERLAP_LABELS)

plot_signal_after_framing(FIG_SIZE, DPI, get_fig_save_path('signal_after_framing_with_smoothing.png'), \
    FRAMING_AUDIO_SEGMENT, FRAMING_FRAMES, FRAME_X_AX_START, FRAME_X_AX_END, FRAME_Y_AX_START, FRAME_Y_AX_END, \
    FRAMING_OVERLAP_TICKS, FRAMING_OVERLAP_LABELS, True)

In [ ]:
def plot_frame_1(audio_segment, frames, y_start, y_end):
    hop_size = FRAMING_HOP_SIZE
    
    f = frames[0]
    plt.plot(f/audio_segment.max())
    audio_ticks = np.arange(0, f.size, int(f.size/8))
    plt.xticks(audio_ticks, ["{:.0f}".format(tick + (0*hop_size)) for tick in audio_ticks], rotation='45')
    plt.xlim([0, f.size])

    plt.ylim([y_start, y_end])

    plt.xlabel('Sample')
    plt.ylabel('Amplitude');
    
plot_frame_1(FRAMING_AUDIO_SEGMENT, FRAMING_FRAMES, FRAME_Y_AX_START, FRAME_Y_AX_END)

In [ ]:
def plot_frame_2(audio_segment, frames, y_start, y_end):
    hop_size = FRAMING_HOP_SIZE
    
    f = frames[1]
    plt.plot(f/audio_segment.max())

    plt.axvspan(0, f.size, color='red', alpha=0.15)

    audio_ticks = np.arange(0, f.size, int(f.size/8))
    plt.xticks(audio_ticks, ["{:.0f}".format(tick + (1*hop_size)) for tick in audio_ticks], rotation='45')
    plt.xlim([0, f.size])

    plt.ylim([y_start, y_end])

    plt.xlabel('Sample')
    plt.ylabel('Amplitude');
    
plot_frame_2(FRAMING_AUDIO_SEGMENT, FRAMING_FRAMES, FRAME_Y_AX_START, FRAME_Y_AX_END)

In [ ]:
def plot_frame_3(audio_segment, frames, y_start, y_end):
    hop_size = FRAMING_HOP_SIZE
    
    f = frames[2]
    plt.plot(f/audio_segment.max())
    audio_ticks = np.arange(0, f.size, int(f.size/8))
    plt.xticks(audio_ticks, ["{:.0f}".format(tick + (2*hop_size)) for tick in audio_ticks], rotation='45')
    plt.xlim([0, f.size])

    plt.ylim([y_start, y_end])

    plt.xlabel('Sample')
    plt.ylabel('Amplitude');
    
plot_frame_3(FRAMING_AUDIO_SEGMENT, FRAMING_FRAMES, FRAME_Y_AX_START, FRAME_Y_AX_END)

### Hann Window

In [ ]:
# widnow function
def plot_hann_window(figsize, dpi, save_path):
    frame_size = 441
    sample_idxs = np.arange(0, frame_size)
    smoothing_coefficients = v_hann(sample_idxs, frame_size)

    plt.figure(figsize=figsize)
    plt.plot(smoothing_coefficients)

    x_ticks = np.arange(0, frame_size, 44, dtype=int)
    plt.xticks(x_ticks, ["{:.0f}".format(tick+1) for tick in x_ticks], rotation='45')
    plt.xlim([0, frame_size])

    plt.xlabel('Sample');
    plt.ylabel('Smoothing Coefficient');

    plt.tight_layout()
    plt.savefig(save_path, dpi=dpi)

    dbg.log('sample values:', v_hann([0, 110 ,220 ,330 ,440], frame_size))
    
plot_hann_window(FIG_SIZE, DPI, get_fig_save_path('hann_window.png'))

In [ ]:
# audio frame (not smoothed)
def plot_frame_before_smoothing(figsize, dpi, save_path):
    frame_size = 441
    frame_start = int(0.5 * FS) # start at 0.5sec
    frame_end = frame_start + frame_size
    frame = AUDIO_BUFFER[frame_start: frame_end]

    plt.figure(figsize=figsize)
    plt.plot(frame / frame.max())

    x_ticks = np.arange(0, frame_size, 44, dtype=int)
    plt.xticks(x_ticks, ["{:.0f}".format(tick+1) for tick in x_ticks], rotation='45')
    plt.xlim([0, frame_size])

    plt.xlabel('Sample');
    plt.ylabel('Amplitude');

    plt.tight_layout()
    plt.savefig(save_path, dpi=dpi)

    x_ax_start, x_ax_end, y_ax_start, y_ax_end = plt.axis()
    return x_ax_start, x_ax_end, y_ax_start, y_ax_end
    
HANN_X_AX_START, HANN_X_AX_END, HANN_Y_AX_START, HANN_Y_AX_END = plot_frame_before_smoothing(FIG_SIZE, DPI, get_fig_save_path('frame_without_smoothing.png'))

In [ ]:
# audio frame (smoothed)
def plot_frame_after_smoothing(figsize, dpi, save_path, x_start, x_end, y_start, y_end):
    frame_size = 441
    frame_start = int(0.5 * FS) # start at 0.5sec
    frame_end = frame_start + frame_size
    frame = AUDIO_BUFFER[frame_start: frame_end]
    sample_idxs = np.arange(0, frame_size)
    smoothing_coefficients = v_hann(sample_idxs, frame_size)
    smoothed_frame = smoothing_coefficients * frame

    plt.figure(figsize=figsize)
    plt.plot(smoothed_frame / frame.max())

    x_ticks = np.arange(0, frame_size, 44, dtype=int)
    plt.xticks(x_ticks, ["{:.0f}".format(tick+1) for tick in x_ticks], rotation='45')
    plt.xlim([x_start, x_end])

    y_ticks = np.arange(-1, 1.5, 0.5)
    plt.yticks(y_ticks)
    plt.ylim([y_start, y_end])

    plt.xlabel('Sample');
    plt.ylabel('Amplitude');

    plt.tight_layout()
    plt.savefig(save_path, dpi=dpi)
    
plot_frame_after_smoothing(FIG_SIZE, DPI, get_fig_save_path('frame_with_smoothing.png'), HANN_X_AX_START, HANN_X_AX_END, HANN_Y_AX_START, HANN_Y_AX_END)

### Time/Frequency Domain Audio Signal Representation

In [ ]:
# spectrogram
def plot_spectrogram(figsize, dpi, save_path, compute_log=False, compute_log_mel=False):
    frame_size = 4096
    hop_size = 441
    sample_rate = FS
    frame_rate = 100
    
    window = get_window('hann', frame_size)
    overlap = frame_size - sample_rate / frame_rate # frame size - hop size
    
    # spectrogram
    frequencies, times, spec = spectrogram(AUDIO_BUFFER, fs=FS, window=window, noverlap=overlap)
    # base 10 log spectrogram
    log_spec = np.log10(spec+1)
    # mel scaled base 10 log spectrogram
    filterbank = LogarithmicFilterbank(frequencies).T #MelFilterbank(frequencies).T
    mel_freq_bins = LogarithmicFilterbank(frequencies).center_frequencies #MelFilterbank(frequencies).center_frequencies
    log_mel_spec = np.log10(np.matmul(filterbank, spec) + 1)
    
    plt.figure(figsize=figsize)
    if(compute_log):
        plt.imshow(log_spec, interpolation=None, origin='lower', aspect='auto')
    elif(compute_log_mel):
        plt.imshow(log_mel_spec, interpolation=None, origin='lower', aspect='auto')
    else:
        plt.imshow(spec, interpolation=None, origin='lower', aspect='auto')
    
    t_tick_idxs = np.arange(0, len(times), 100, dtype=int)
    t_tick_lbls = ["{:.1f}".format(tick/100) for tick in t_tick_idxs]
    f_tick_idxs = np.arange(0, len(frequencies), int(len(frequencies)/8))
    f_tick_lbls = ["{:.1f}".format(tick) for tick in frequencies[f_tick_idxs]]
    if(compute_log_mel):
        f_tick_idxs = np.append(np.arange(0, len(mel_freq_bins), 10), len(mel_freq_bins)-1)
        f_tick_lbls = ["{:.1f}".format(tick) for tick in mel_freq_bins[f_tick_idxs]]
    plt.xticks(t_tick_idxs, t_tick_lbls, rotation='45')
    plt.yticks(f_tick_idxs, f_tick_lbls)
    
    plt.xlabel('Time (seconds)');
    plt.ylabel('Frequency (Hz)');
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=dpi)
    
    dbg.log((AUDIO_BUFFER.size - frame_size) / 441)
    dbg.log(times.shape)
    dbg.log(frequencies.size)
    dbg.log(spec.shape)
    
    dbg.log(filterbank.shape)
    dbg.log(mel_freq_bins.shape)

plot_spectrogram(FIG_SIZE, DPI, get_fig_save_path('spectrogram.png'))
plot_spectrogram(FIG_SIZE, DPI, get_fig_save_path('log_spectrogram.png'), compute_log=True)
plot_spectrogram(FIG_SIZE, DPI, get_fig_save_path('log_filtered_spectrogram.png'), compute_log_mel=True)

### Mel Scale

In [ ]:
def plot_mel_scale(figsize, dpi, save_path):
    frame_size = 4096
    hop_size = 441
    sample_rate = FS
    frame_rate = 100
    
    window = get_window('hann', frame_size)
    overlap = frame_size - sample_rate / frame_rate # frame size - hop size
    
    # mel scale
    frequencies, _, _= spectrogram(AUDIO_BUFFER, fs=FS, window=window, noverlap=overlap)
    filterbank = LogarithmicFilterbank(frequencies).T #MelFilterbank(frequencies).T
    mel_freq_bins = LogarithmicFilterbank(frequencies).center_frequencies #MelFilterbank(frequencies).center_frequencies
    
    plt.figure(figsize=figsize)
    plt.imshow(filterbank, interpolation=None, origin='lower', aspect='auto')
    
    #debug
    #mel_idx = 39
    #dbg.log(filterbank.shape, frequencies.shape)
    #dbg.log(filterbank[39][filterbank[39] != 0 ])
    #none_zero_idxs = np.where([filterbank[mel_idx] != 0 ])[1]
    #dbg.log(none_zero_idxs)
    #dbg.log(filterbank[mel_idx][none_zero_idxs])
    
    f_tick_idxs = np.arange(0, len(frequencies), int(len(frequencies)/8))
    f_tick_lbls = ["{:.1f}".format(tick) for tick in frequencies[f_tick_idxs]]
    mel_f_tick_idxs = np.append(np.arange(0, len(mel_freq_bins), 10), len(mel_freq_bins)-1)
    mel_f_tick_lbls = ["{:.1f}".format(tick) for tick in mel_freq_bins[mel_f_tick_idxs]]
    plt.xticks(f_tick_idxs, f_tick_lbls, rotation='45')
    plt.yticks(mel_f_tick_idxs, mel_f_tick_lbls)
    
    plt.xlabel('Hertz scale');
    plt.ylabel('Mel Scale');
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=dpi)
    
plot_mel_scale(FIG_SIZE, DPI, get_fig_save_path('logarithmic_filterbank.png'))

### Decibel Scale